In [3]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import statsmodels.api as sm
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path
import numpy as np

In [4]:
theses_df = pd.read_csv('theses_v2.csv')
theses_df

,Unnamed: 0,title,school,department,degree,submitting_type,language,subfield,research_field,keyword,committee_chair,committee_members,graduation_year,partnering_agency,abstract_length,abstract
0,0,Does Inflammation Damage White Matter ...,Emory College,Neuroscience and Behavioral Biology,B.S.,Honors Thesis,English,NaN,"Health Sciences, Mental Health; Psychology, Ps...",Neurobiology; Depression; DTI Imaging,Boadie Dunlop,Kate O'Toole;Micheal Crutcher,2020,NaN,2451,Background: Major depressive disorder (MDD) is...
1,1,Chlorhexidine bigluconate resistance in Met...,Rollins School of Public Health,Epidemiology,M.P.H.,Master's Thesis,English,Epidemiology - MPH & MSPH,"Health Sciences, Epidemiology; Biology, Microb...",MRSA; Antimicrobial Resistance,Scott Fridkin,Sarah Satola,2020,NaN,2283,Background: With increased chlorhexidine use i...
2,2,IRF4 controls cell fate outcomes during the...,Laney Graduate School,Biological and Biomedical Sciences,Ph.D.,Dissertation,English,Genetics and Molecular Biology,"Health Sciences, Immunology; Biology, Cell; Bi...",Plasma cell; IRF4; B cell; MYC; L-selectin; CD...,Joshy Jacob;Roger B. Deal;Jeremy M. Boss;Willi...,NaN,2021,NaN,3042,Naive B cell (nB) differentiation to antibody-...
3,3,From Surviving to Thriving: Exploring Healt...,Candler School of Theology,Ministry,DMin,Dissertation,English,NaN,Theology,Succession,Jennifer Ayres,Teresa Fry Brown,2021,NaN,1092,This project focuses on developing strategic p...
4,4,Maternal Exposures to Cigarette Smoking and ...,Rollins School of Public Health,Epidemiology,M.P.H.,Master's Thesis,English,Epidemiology - MPH & MSPH,"Health Sciences, Public Health; Health Science...",case-control study; alcohol; birth defects; ep...,Vijaya Kancherla;Paul Romitti,NaN,2020,"University, college or education institution (...",2606,BACKGROUND: Congenital diaphragmatic hernia (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9976,9976,Zimbabwe's Maturing HIV Epidemic: A Triangulat...,Rollins School of Public Health,Global Health,MPH,Master's Thesis,English,NaN,"Health Sciences, Public Health",Zimbabwe; HIV; AIDS; Data triangulation; human...,Roger Rochat,NaN,2015,"International governmental organization (e.g.,...",1955,Objective: To assess Zimbabwe's HIV prevalence...
9977,9977,Zooarchaeological Faunal Identifiability: Usin...,Emory College,Anthropology,B.S.,Honors Thesis,English,NaN,"Anthropology, Archaeology",GIS; Faunal Identifiability; Zooarchaeology,Jessica Thompson;Michael Page,John Lindo;Melvin Konner,2019,NaN,2160,Zooarchaeological analysis aims to identify th...
9978,9978,Zoomorphic Others: The Animalization of Stigma...,Laney Graduate School,English and Creative Writing,MA,Master's Thesis,English,NaN,"Literature, General; Literature, Modern",animality; criminality; zoomorphism; animaliza...,Laura Otis,Christina Marie Colvin;Sari B Altschuler,2016,NaN,1584,"Zoomorphism, the act of portraying humans with..."
9979,9979,Zoonotic Diseases in Refugee or Internally Dis...,Rollins School of Public Health,Global Health,MPH,Master's Thesis,English,NaN,"Agriculture, Animal Pathology; Health Sciences...",Zoonotic Disease; Conflict; Refugee Camp; Camp...,Stephanie Salyer,Casey Barton-Behravesh;Carlos Navarro Colorado,2017,CDC,2238,Introduction: Emerging and re-emerging zoonoti...


In [5]:
file = open('dissertation_betweenness', 'rb')
betweenness = pickle.load(file)
file.close()


In [6]:
path_to_mallet = '../../mallet-2.0.8/bin/mallet'
num_topics = 25

output_directory_path = 'topic-model-output'
path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/mallet.topic_distributions.{str(num_topics)}"
path_to_diagnostics             = f"{output_directory_path}/diagnostics.xml"
path_to_word_weights            = f"{output_directory_path}/word.weights"

topic_dist_list = little_mallet_wrapper.load_topic_distributions(path_to_topic_distributions)

In [14]:
# title_list = theses_df['title'].tolist()
topic_dist_list_in_order = []
betweenness_in_order = []
for idx, row in theses_df.iterrows():
    title = row['title'].strip()
    
    if title not in betweenness: continue
        
    topic_dist_list_in_order.append(topic_dist_list[idx])
    betweenness_in_order.append(betweenness[title])

print(len(topic_dist_list_in_order))
print(len(betweenness_in_order))

9852
9852


In [16]:
x, y = np.array(topic_dist_list_in_order), np.array(betweenness_in_order)
x = sm.add_constant(x)

In [17]:
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     4.673
Date:                Sun, 12 Dec 2021   Prob (F-statistic):           2.92e-13
Time:                        02:38:45   Log-Likelihood:                 58936.
No. Observations:                9852   AIC:                        -1.178e+05
Df Residuals:                    9827   BIC:                        -1.176e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002   1.09e-05     21.956      0.0

In [19]:
sorted(betweenness.items(), key=lambda x: x[1], reverse=True)

[('Investigating heterogeneity in the dynamics of virus and immune response following a yellow fever vaccination',
  0.009955076454019274),
 ('The consequences of malnutrition, and effects of multiple micronutrient powders, on young child development in India',
  0.00980713989479091),
 ('U.S. Attitudes toward Adolescent Sexual Activity: A Feminist\nEngagement with the Second Demographic Transition',
  0.009794129030263398),
 ('Explaining the decline of coronary heart disease mortality in\nthe United States in the 1960s: An historical analysis',
  0.007939942074236562),
 ('Sociocultural Factors Shaping Trust and Teamwork among Community\nMaternal and Newborn Health Workers in Rural Ethiopia',
  0.007790750726469232),
 ('Senior Honors Recital', 0.0072399689611134825),
 ('The interaction between Streptococcus pneumoniae and\nother respiratory pathogens, including viruses and bacteria\ncommonly colonizing the nasopharynx',
  0.006768179314010212),
 ('HIV Surveillance, Community Engagement,